In [164]:
import numpy as np
from scipy.stats import norm

# Pricing with closed formula

In [165]:
def d_p(x,v):
    return (np.log(x) + v * v / 2) / v

def d_m(x,v):
    return (np.log(x) - v * v / 2) / v

def libor_price(L_i, K, H, v_i):
    return L_i * ( norm.cdf(d_p(L_i / K, v_i)) - norm.cdf(d_p(L_i / H, v_i)) ) - \
            K * ( norm.cdf(d_m(L_i / K, v_i)) - norm.cdf(d_m(L_i / H, v_i)) ) - \
            H * ( norm.cdf(d_p(H * H / K / L_i, v_i)) - norm.cdf(d_p(H / L_i, v_i)) ) + \
            K * L_i / H * ( norm.cdf(d_m(H * H / K / L_i, v_i)) - norm.cdf(d_m(H / L_i, v_i)))

In [166]:
d_m(0.13 / 0.001, 3 * 0.25)

6.115045933940777

In [167]:
libor_price(0.13, 0.01, 0.28, 3 * 0.25)

0.06571345192162645

# Derivative calculation

We calculate the derivative by formula and we check it with the approximation

In [168]:
def d_prime(x,v):
    return 1.0 / (x * v)

def dF_dL(L_i, K, H, v_i):
    val =  norm.cdf(d_p(L_i / K, v_i)) - norm.cdf(d_p(L_i / H, v_i)) + \
            L_i * ( norm.pdf(d_p(L_i / K, v_i)) * d_prime(L_i / K, v_i) / K - \
                   norm.pdf(d_p(L_i / H, v_i)) * d_prime(L_i / H, v_i) / H ) - \
            K * ( norm.pdf(d_m(L_i / K, v_i)) * d_prime(L_i / K, v_i) / K - \
                 norm.pdf(d_m(L_i / H, v_i)) * d_prime(L_i / H, v_i) / H) - \
            H * ( -norm.pdf(d_p(H * H / K / L_i, v_i)) * d_prime(H * H / K / L_i, v_i) * H * H / K / L_i / L_i + \
                        norm.pdf(d_p(H / L_i, v_i)) * H / L_i / L_i ) * d_prime(H / L_i, v_i) + \
            K / H * ( norm.cdf(d_m(H * H / K / L_i, v_i)) - norm.cdf(d_m(H / L_i, v_i)) ) + \
            K * L_i / H * ( -norm.pdf(d_m(H * H / K / L_i, v_i)) * H * H / K / L_i / L_i * d_prime(H * H / K / L_i, v_i) + \
                          norm.pdf(d_m(H / L_i, v_i)) * H / L_i / L_i * d_prime(H / L_i, v_i) )
    return val

In [169]:
dF_dL(0.13, 0.01, 0.28, 3 * 0.25)

-0.08090766447073611

In [170]:
delta = 10e-6
(libor_price(0.13 + delta, 0.01, 0.28, 3 * 0.25) - libor_price(0.13, 0.01, 0.28, 3 * 0.25)) / delta 

-0.08095125657026081

# We've got the same values!

# Now we make a simulation

In [173]:
def bern(p):
    if np.random.rand() < p:
        return -1.0
    else:
        return 1.0
    
def pos_part(x):
    if x > 0:
        return x
    else:
        return 0.0
    
def one_simulation(L_i, K, H, sigma, h,  i = 9):
    
    M = int(i / h)
    k = 0
    lnL = np.log(L_i)
    Z = 0.0
    sqh = np.sqrt(h)
    v_i = sigma * np.sqrt(i)
    
    lnH = np.log(H)
    lambda_sqh =  -0.5 * sigma * sigma * h + sigma * sqh
    
    value_to_check = lnH - lambda_sqh
    #print("vtc", value_to_check)
    while(k < M):
        #print('# ', k)
        
        if lnL >= value_to_check:
            '''
            if bern(1 - lambda_sqh / (lnH - lnL + lambda_sqh)) < 0:
                #print("1")
                lnL = lnL - lambda_sqh
            else:
                #print("2")
                return Z
            '''
            return Z
        
        #print("!!")   
        eta = bern(0.5)
        #print(dF_dL(np.exp(lnL), K, H, v_i))
        Z = Z - sigma * dF_dL(np.exp(lnL), K, H, v_i) * sqh * eta
        lnL = lnL - 0.5 * sigma * sigma * h + sigma * sqh * eta
        k += 1
        #print(k * h, lnL, Z)
        
    #return k == M, lnL, Z

    #print("3")
    val = pos_part(np.exp(lnL) - K) + Z
    #print(val)
    
    return val



In [175]:
s = 0.0
num_sim = 1000
for i in range(num_sim):
    s += one_simulation(0.13, 0.01, 0.28, 0.25, 0.1,  i = 9)
    
    if (i > 1 and i % 100 == 0):
        print(i, s / i)
print(s / num_sim)

100 0.05597028617340327
200 0.06298914397277777
300 0.06439126365426591
400 0.07107580869364616
500 0.06186716186251829
600 0.0621488402863337
700 0.06124915223312292
800 0.05617827344948874
900 0.05607824331796241
0.051529085128700816


# We plot the results of c++ simulation

In [148]:
import matplotlib.pyplot as plt

In [162]:
steps = [0.05, 0.1, 0.125, 0.2, 0.25]
alg1 = [0.0657638, 0.0657759, 0.0650573, 0.0662222, 0.066845 ]
alg2 = [0.0639383, 0.0625295, 0.0619019, 0.0624662, 0.0626165 ]
trues = [0.0657134, 0.0657134, 0.0657134, 0.0657134, 0.0657134]

plt.plot(steps, alg1,  "go--", label = "alg1")
plt.plot(steps, alg2, "ro--", label = "alg2")
plt.plot(steps, trues, c = "blue", label = "ground_truth")
plt.legend()
plt.xlabel("h")
plt.ylabel("price")
plt.title("1000000 simulations")
plt.savefig("1000000plot.png")

'''
******ALGORITHM 1********
step: 0.05, result: 0.0657638, duration: 415.728s
step: 0.1, result: 0.0657759, duration: 208.507s
step: 0.125, result: 0.0650573, duration: 166.505s
step: 0.2, result: 0.0662222, duration: 103.779s
step: 0.25, result: 0.066845, duration: 83.054s
******ALGORITHM 2********
step: 0.05, result: 0.0639383, duration: 410.521s
step: 0.1, result: 0.0625295, duration: 204.981s
step: 0.125, result: 0.0619019, duration: 163.727s
step: 0.2, result: 0.0624662, duration: 101.681s
step: 0.25, result: 0.0626165, duration: 81.186s
'''

'\n******ALGORITHM 1********\nstep: 0.05, result: 0.0657638, duration: 415.728s\nstep: 0.1, result: 0.0657759, duration: 208.507s\nstep: 0.125, result: 0.0650573, duration: 166.505s\nstep: 0.2, result: 0.0662222, duration: 103.779s\nstep: 0.25, result: 0.066845, duration: 83.054s\n******ALGORITHM 2********\nstep: 0.05, result: 0.0639383, duration: 410.521s\nstep: 0.1, result: 0.0625295, duration: 204.981s\nstep: 0.125, result: 0.0619019, duration: 163.727s\nstep: 0.2, result: 0.0624662, duration: 101.681s\nstep: 0.25, result: 0.0626165, duration: 81.186s\n'